In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import torch
from utils.Gridworld import Gridworld
from tqdm import tqdm
from utils.miscellaneous import *
from collections import deque
import random

In [ ]:
# Test the game
game = Gridworld(size=4, mode='static')
print(game.display())
game.reward()
action_set = {
0: 'u',
1: 'd',
2: 'l',
3: 'r',
}


In [ ]:
# Define the Q network 
l1 = len(game.board.render_np().flatten())
l2, l3 = 150, 100
l4 = game.board.size

gamma = 0.9
epsilon = 1.0
learning_rate = 1e-3

model = torch.nn.Sequential(
torch.nn.Linear(l1, l2),
torch.nn.ReLU(),
torch.nn.Linear(l2, l3),
torch.nn.ReLU(),
torch.nn.Linear(l3,l4)
)
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Train loop
epochs = 5000
losses = []
for i in tqdm(range(epochs)):
    game = Gridworld(size=4, mode='player')
    state_ = game.board.render_np().flatten() \
             + np.random.rand(1,64)/10.0
    
    state1 = torch.from_numpy(state_).float()
    game_on = 1
    while(game_on):
        qval = model(state1)
        qval_ = qval.data.numpy()
        if np.random.rand() < epsilon:
            action_ = np.random.randint(0, 4)
        else:
            action_ = np.argmax(qval_)
        
        action = action_set[action_]
        game.makeMove(action)
        # Get new state and reward to the action made
        state2_ = game.board.render_np().flatten() \
             + np.random.rand(1,64)/10.0
        state2 = torch.from_numpy(state2_).float()
        reward = game.reward()

        with torch.no_grad():
            newQ = model(state2)
        maxQ = torch.max(newQ)
        if reward == -1:
            Y = reward + (gamma * maxQ)

        else:
            Y = reward

        Y = torch.Tensor([Y,]).detach()
        X = qval[0:1,action_]

        loss = loss_fn(X, Y)
        optimizer.zero_grad()
        loss.backward()
        losses.append(loss.item())
        optimizer.step()
        state1 = state2
        if reward != -1:
            game_on = 0
        if epsilon > 0.1:
            epsilon -= (1/epochs)


In [ ]:
smooth_losses = moving_average(losses, w=500)
plt.plot(smooth_losses)
plt.show()

## Exprerience replay 
This network is not able to learn the 'random' mode of the game. This is mostly due to catastrophic forgetting, in which learning from a single experience may overwrite previously learned action-state associations.

In [ ]:
# Train loop
epochs = 5000
mem_size = 1000
batch_size = 200
replay = deque(maxlen=mem_size)
max_moves = 50 # max number of moves before game over
h = 0

losses = []
for i in tqdm(range(epochs)):
    game = Gridworld(size=4, mode='random')
    state1_ = game.board.render_np().flatten() \
             + np.random.rand(1,64)/10.0
    
    state1 = torch.from_numpy(state1_).float()
    game_on = 1
    mov = 0
    while(game_on):
        mov += 1
        qval = model(state1)
        qval_ = qval.data.numpy()
        if np.random.rand() < epsilon:
            action_ = np.random.randint(0, 4)
        else:
            action_ = np.argmax(qval_)
        
        action = action_set[action_]
        game.makeMove(action)
        # Get new state and reward to the action made
        state2_ = game.board.render_np().flatten() \
             + np.random.rand(1,64)/10.0
        state2 = torch.from_numpy(state2_).float()
        reward = game.reward()
        done = True if reward > 0 else False

        # Create and save the expericne 
        exp = (state1, action_, reward, state2, done)
        replay.append(exp)
        state1 = state2

        if len(replay) > batch_size:
            minibatch = random.sample(replay, batch_size)

            state1_batch = torch.cat([s1 for (s1,a,r,s2,d) in minibatch]) 
            action_batch = torch.Tensor([a for (s1,a,r,s2,d) in minibatch]) 
            reward_batch = torch.Tensor([r for (s1,a,r,s2,d) in minibatch])
            state2_batch = torch.cat([s2 for (s1,a,r,s2,d) in minibatch]) 
            done_batch = torch.Tensor([d for (s1,a,r,s2,d) in minibatch])

            Q1 = model(state1_batch)
            with torch.no_grad():
                Q2 = model(state2_batch)

            Y = reward_batch + gamma * ((1 - done_batch) * torch.max(Q2,dim=1)[0])
            X = Q1.gather(dim=1,index=action_batch.long().unsqueeze(dim=1)).squeeze()

            loss = loss_fn(X, Y.detach())
            optimizer.zero_grad()
            loss.backward()
            losses.append(loss.item())
            optimizer.step()

        if reward != -1 or mov > max_moves:
            status = 0
            mov = 0


losses = np.array(losses)

In [ ]:
smooth_losses = moving_average(losses, w=500)
plt.plot(smooth_losses)
plt.show()